# Introduction to Cloud Machine Learning with Flask API and CNTK



In [1]:
#load libraries
import os,sys
import pkg_resources
from flask import Flask, render_template, request, send_file
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import wget
import numpy as np
from PIL import Image, ImageOps
from urllib.request import urlretrieve
from cntk import load_model, combine
import mxnet as mx

print("System version: {}".format(sys.version))
print("Flask version: {}".format(pkg_resources.get_distribution("flask").version))

System version: 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Flask version: 0.12


In [ ]:
app = Flask(__name__)

In [ ]:
# Open downloaded labels (1000)
urlretrieve("http://data.mxnet.io/models/imagenet/synset.txt", "resnet_152_cntk.txt")

with open('resnet_152_cntk.txt', 'r') as f:
    LABELS = [l.rstrip() for l in f]

print("Label length: ", len(LABELS))


In [ ]:
def get_image(url):
    return urlretrieve(url)[0]

In [ ]:
def predict(filename, synsets):
    
    # Load image and resize    
    img = Image.open(filename)
    # Centre crop
    img = ImageOps.fit(img, (224, 224), Image.ANTIALIAS)
    
    # Show image
    plt.imshow(img)
    plt.axis('off')
    
    # Do we need to subtract mean? Or batch normalisation layer takes cares of this???
    
    img = np.array(img, dtype=np.float32)
    img = np.ascontiguousarray(np.transpose(img, (2, 0, 1)))
    
    # Use last layer to make prediction
    z_out = combine([z.outputs[3].owner])
    result = np.squeeze(z_out.eval({z_out.arguments[0]:[img]}))

    # Sort probabilities 
    a = np.argsort(result)[-5:]  
    
    #predicted_category = " ".join(synsets[a].split(" ")[1:])
    for pos in a:
        predicted_category = " ".join(synsets[pos].split(" ")[1:])
        print(predicted_category)

In [ ]:
# Load CNTK Model
model_path = '/datadrive/installer/cntk/Examples/Image/PretrainedModels/ResNet_152.model'
z = load_model(model_path)

In [ ]:
predict(get_image("http://writm.com/wp-content/uploads/2016/08/Cat-hd-wallpapers.jpg"), LABELS)

In [ ]:
predict(get_image("https://migonzastorage.blob.core.windows.net/deep-learning/neko.jpg"), LABELS)

In [ ]:
predict(get_image("https://upload.wikimedia.org/wikipedia/commons/1/1e/Bald_Eagle_Portrait.jpg"), LABELS)

In [ ]:
def predict(image, n_results, model = 'cntk_resnet152'):
    if 

In [13]:
class Model(object):
    def __new__(cls, *args, **kwargs):
        if cls is Model:                            # <-- required because Line's
            base_library, args = args[0], args[1:]   #     __new__ method is the
            if base_library == 'mxnet':          #     same as Shape's
                new_cls = MXNetModel
            else:
                raise ValueError("Invalid base library: {}.".format(base_library))
        else:
            new_cls = cls
        return super(Model, cls).__new__(new_cls, *args, **kwargs)
    
    def get_library(self):
        return "No base library"

class MXNetModel(Model):
    def get_library(self):
        return "MXNet"

class OtherModel(Model):
    pass

In [14]:
mm = OtherModel()
mm.get_library()

'No base library'